In [19]:
import pandas
import math
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import norm
from statsmodels.sandbox.regression.gmm import GMM
from statsmodels.base.model import GenericLikelihoodModel

In [20]:
data2 = np.genfromtxt('data2.dat', delimiter='  ')
data2 = data2.transpose()

y = data2[0]
x = sm.add_constant(data2[1])

In [21]:
#part a - MLE


class part_b(GenericLikelihoodModel):
    """class for evaluating question 2 part a"""
    
    def nloglikeobs(self, params):
        t1, t2, sigma = params
        endog, exog = self.endog, self.exog.squeeze()
        eps = np.log(endog) - t1 - exog**t2
        #calculate log likelihood
        return -norm(0,sigma).logpdf(eps).sum()

    
    def fit(self, start_params=None, maxiter=10000, maxfun=5000, **kwds):
        if start_params == None:
            start_params = [.5, .5,.5]
        return super(part_b, self).fit(start_params=start_params,
                                       maxiter=maxiter, maxfun=maxfun, **kwds)

    
model_b = part_b(data2[0],data2[1])
result_b = model_b.fit()
print(result_b.summary(xname=['theta_1', 'theta_2', 'sigma']))


#sources (same as last time):
#http://www.statsmodels.org/0.6.1/examples/notebooks/generated/generic_mle.html
#http://rlhick.people.wm.edu/posts/estimating-custom-mle.html

Optimization terminated successfully.
         Current function value: 1.381592
         Iterations: 202
         Function evaluations: 360
                                part_b Results                                
Dep. Variable:                      y   Log-Likelihood:                -138.16
Model:                         part_b   AIC:                             278.3
Method:            Maximum Likelihood   BIC:                             280.9
Date:                Thu, 27 Sep 2018                                         
Time:                        17:35:31                                         
No. Observations:                 100                                         
Df Residuals:                      99                                         
Df Model:                           0                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------

In [22]:
#part c - GMM

class part_c(GMM):
    """class for evaluating question 2 part b"""
    
    def __init__(self, *args, **kwds):
        # set appropriate counts for moment conditions and parameters
        kwds.setdefault('k_moms', 2)
        kwds.setdefault('k_params',2)
        super(part_c, self).__init__(*args, **kwds)
    
    
    def fit(self, start_params=None, maxiter=10000, **kwds):
        if start_params == None:
            start_params = np.array([.5, .5])
        return super(part_c, self).fit(start_params=start_params,
                                       maxiter=maxiter,  **kwds)
    
        
    def momcond(self, params):
        t1,t2 = params  
        endog, exog = self.endog, self.exog.squeeze()
        #different model, essentially the same since its an injective mapping
        eps =  np.log(endog) - t1 - exog**t2 
        g = np.column_stack( (eps, eps*exog ))
        return g 

    
model_c = part_c(data2[0],data2[1], None)
result_c = model_c.fit(maxiter=2, optim_method='nm', wargs=dict(centered=False))
print(result_c.summary(xname=['theta_1', 'theta_2']))


#sources (same as last time):
#https://github.com/josef-pkt/misc/blob/master/notebooks/ex_gmm_gamma.ipynb
#https://www.statsmodels.org/dev/generated/statsmodels.sandbox.regression.gmm.GMM.html#statsmodels.sandbox.regression.gmm.GMM
#https://gist.github.com/josef-pkt/6895915

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 61
         Function evaluations: 119
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 23
         Function evaluations: 46
                                part_c Results                                
Dep. Variable:                      y   Hansen J:                    1.456e-09
Model:                         part_c   Prob (Hansen J):                   nan
Method:                           GMM                                         
Date:                Thu, 27 Sep 2018                                         
Time:                        17:35:31                                         
No. Observations:                 100                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
theta_1     